<a href="https://colab.research.google.com/github/Amitukra/Data-Analysis-Process/blob/main/Data%20extraction/Untitled9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import requests as re
from bs4 import BeautifulSoup

In [ ]:
df = pd.read_csv("/content/GG - Sheet1.csv")

In [ ]:
df.head()

,Business Name,Email,Phone,Address,Website,Category Name
0,Webterior Designs,hello@webteriordesigns.com,"(844) 917-1836, (844)917-1836",2084 E 8th St,webteriordesigns.com,Marketing agency
1,"Media Explode Marketing, Web Development, Prin...","support@mediaexplode.com, marketing@mediaexplo...","18887788160, 12127442070, 2127442070",2271 E 14th St,mediaexplode.com,Marketing agency


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
import re

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def get_about_us_content(website):
    base_url = f"https://{website.strip('/')}/"
    paths = ["about", "about-us", "company/about", "who-we-are"]

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"
                      " (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"
    }

    for path in paths:
        try:
            full_url = urljoin(base_url, path)
            response = requests.get(full_url, headers=headers, timeout=10)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, "html.parser")
                paragraphs = soup.find_all(['p', 'div'], text=True)
                text = ' '.join(p.get_text(strip=True) for p in paragraphs)
                clean_text = re.sub(r'\s+', ' ', text).strip()
                if len(clean_text) > 100:
                    return clean_text
        except:
            continue
    return "Not found"

df["About Content"] = df["Website"].apply(get_about_us_content)

# -------------------------------
# STEP 3: Extract Keywords (TF-IDF)

def extract_keywords(text, top_n=10):
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform([text])
    scores = zip(vectorizer.get_feature_names_out(), tfidf_matrix.toarray()[0])
    sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)
    keywords = [word for word, score in sorted_scores[:top_n]]
    return keywords

df["Keywords"] = df["About Content"].apply(lambda x: extract_keywords(x) if x != "Not found" else [])

# -------------------------------
# STEP 4: Generate Content from Keywords (Basic Version)

def generate_summary(keywords):
    if not keywords:
        return "No content available."
    return f"Our brand focuses on {', '.join(keywords[:5])}, delivering quality and innovation in every step."

df["Generated Content"] = df["Keywords"].apply(generate_summary)

# -------------------------------
# RESULT
print(df[["Business Name", "Website", "Keywords", "Generated Content"]])

<ipython-input-38-6d20991552de>:16: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  paragraphs = soup.find_all(['p', 'div'], text=True)


                                       Business Name               Website  \
0                                  Webterior Designs  webteriordesigns.com   
1  Media Explode Marketing, Web Development, Prin...      mediaexplode.com   

                                            Keywords  \
0  [data, service, information, privacy, policy, ...   
1  [media, explode, team, business, clients, prid...   

                                   Generated Content  
0  Our brand focuses on data, service, informatio...  
1  Our brand focuses on media, explode, team, bus...  


In [ ]:
df.head()

,Business Name,Email,Phone,Address,Website,Category Name,About Content,Keywords,Generated Content
0,Webterior Designs,hello@webteriordesigns.com,"(844) 917-1836, (844)917-1836",2084 E 8th St,webteriordesigns.com,Marketing agency,[wpseo_breadcrumb] About What makes a company ...,"[data, service, information, privacy, policy, ...","Our brand focuses on data, service, informatio..."
1,"Media Explode Marketing, Web Development, Prin...","support@mediaexplode.com, marketing@mediaexplo...","18887788160, 12127442070, 2127442070",2271 E 14th St,mediaexplode.com,Marketing agency,(888) 778-8160 Here at Media Explode we believ...,"[media, explode, team, business, clients, prid...","Our brand focuses on media, explode, team, bus..."


In [ ]:
df

,Business Name,Email,Phone,Address,Website,Category Name,About Content,Keywords,Generated Content
0,Webterior Designs,hello@webteriordesigns.com,"(844) 917-1836, (844)917-1836",2084 E 8th St,webteriordesigns.com,Marketing agency,Not found,[],No content available.
1,"Media Explode Marketing, Web Development, Prin...","support@mediaexplode.com, marketing@mediaexplo...","18887788160, 12127442070, 2127442070",2271 E 14th St,mediaexplode.com,Marketing agency,Not found,[],No content available.
